In [3]:
import pandas as pd # 테이블 형식으로 데이터를 처리할 수 있게 하는 라이브러리

In [4]:
from selenium import webdriver as wd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

In [5]:
class Covid:
    def __init__(self, url, loop):
        self.url = url
        self.loop = loop
        self.df = pd.DataFrame()
        self.driver = None
        self.table = {}
        
    def setup(self, path):
        self.driver = wd.Chrome(executable_path=path) # 웹드라이버 경로를 executable_path에 설정
        self.driver.get(self.url)
        
    def move_next(self):
        
        time.sleep(2)
        try:
            move_button_xpath = '//*[@id="DataTables_Table_0_next"]'
            button = self.driver.find_element_by_xpath(move_button_xpath)
            ActionChains(self.driver).move_to_element(button).click(button).perform() 
        
        except: #스크롤해서 버튼 찾기
            move_button_xpath = '/html/body/div[2]/div[2]/div[2]/div/div[5]/div[2]/div[4]/div[3]/div/div[2]/div[5]/a[2]'
            button = self.driver.find_element_by_xpath(move_button_xpath)
            ActionChains(self.driver).move_to_element(button).click(button).perform() 
        
    def get_table(self):
        
        try: # 현재 데이터 조회
            target_table_xpath = '//*[@id="DataTables_Table_0"]'
            table = self.driver.find_element_by_xpath(target_table_xpath)
            
            #xpath를 찾았는데 데이터가 나오지 않는 경우(과거 데이터를 조회하는데 위 xpath는 true이며 데이터가 나오지 않는 경우)
            if table.text == '':
                target_table_xpath = '/html/body/div[2]/div[2]/div[2]/div/div[5]/div[2]/div[4]/div[3]/div/div[2]'
                table = self.driver.find_element_by_xpath(target_table_xpath)
            
            sets = [table.text][0].split('\n')
            
            if 'Previous' in sets[-1]:
                del sets[-1]
            
            self.table['table_name'] = sets[0].split(' ')[1:]
            self.table['idx'] = [sets[i] for i in range(1, len(sets),2)]
            self.table['data'] = [sets[i] for i in range(2, len(sets), 2)]
        
        except: # 과거 데이터 조회
            target_table_xpath = '/html/body/div[2]/div[2]/div[2]/div/div[5]/div[2]/div[4]/div[3]/div/div[2]'
            table = self.driver.find_element_by_xpath(target_table_xpath)

            sets = [table.text][0].split('\n')
            
            
            if 'Previous' in sets[-1]:
                del sets[-1]

            self.table['table_name'] = sets[0].split(' ')[1:]
            self.table['idx'] = [sets[i] for i in range(1, len(sets),2)]
            self.table['data'] = [sets[i] for i in range(2, len(sets), 2)]
        
    def make_table(self):
        # 테이블로 만둘어주는 함수
        df_tmp = pd.DataFrame()
        
        for x in range(1, self.loop + 1):
            
            Covid.get_table(self)
            
            data_tmp = [i.split(' ') for i in self.table['data']]
            # 공백 기준으로 나누면 하나의 데이터로 들어가야할 값이 나누어져서 들어가는 문제를 해결하기 위한 for loop
            for i in data_tmp:
                i.insert(4, ''.join(i[4:-1]))
                del i[5:-1]

            self.table.update({'data' : data_tmp})

            columns = self.table['table_name']
            index = self.table['idx']
            self.df = pd.DataFrame(data_tmp, columns=columns, index=index).reset_index().rename(columns={'index':'연번'})
        
            df_tmp = pd.concat([df_tmp, self.df], axis=0)
            
            Covid.move_next(self)
            
            time.sleep(2)
            
        self.driver.quit() 
        
        return df_tmp.reset_index(drop=True)
        
        
        
    
class Current(Covid):
    def __init__(self, url, loop):
        super().__init__(url, loop)
        self.xpath = {
            'scroll_corona' : '//*[@id="container"]/div[2]/div/div[4]/ul/li[1]/button',
            'select_location' : '//*[@id="status_searchArea"]/option[3]',
            'search_button' : '//*[@id="status_searchButton"]'
        }

    def scroll_covid_table(self):
        self.driver.find_element_by_xpath(self.xpath['scroll_corona']).click()
        
    def select_location(self):
        self.driver.find_element_by_xpath(self.xpath['select_location']).click()
        
    def start_search(self):
        self.driver.find_element_by_xpath(self.xpath['search_button']).click()
        time.sleep(1)
        

class Old(Covid):
    def __init__(self, url, loop):
        super().__init__(url, loop)
        self.xpath = {
            'scroll_corona' : '//*[@id="container"]/div[2]/div/div[4]/ul/li[1]/button',
            'click_old' : '//*[@id="move-cont1"]/div[2]/div[4]/div[1]/ul/li[2]/button',
            'select_location' : '//*[@id="status_pre_searchArea"]/option[3]',
            'search_button' : '//*[@id="status_pre_searchButton"]'
        }

    def scroll_covid_table(self):
        self.driver.find_element_by_xpath(self.xpath['scroll_corona']).click()
        self.driver.find_element_by_xpath(self.xpath['click_old']).click()
        
    def select_location(self):
        self.driver.find_element_by_xpath(self.xpath['select_location']).click()
        
    def start_search(self):
        self.driver.find_element_by_xpath(self.xpath['search_button']).click()

**chromedriver_path는 설치한 웹드라이버 경로로 변경**  
**loop로 얼마만큼의 페이지를 가져올것인지 설정 (최대 27페이지, 한 페이지당 100개 row)**

In [6]:
url = "https://www.seoul.go.kr/coronaV/coronaStatus.do"
chromedriver_path = "C:/chromedriver.exe" #크롬 웹드라이버 경로

### 현재 확진자 현황 (80001번 ~ 현재) 

In [7]:
loop = 2 #반복횟수(얼마만큼의 페이지 데이터를 가져올건지 선택)

current = Current(url = url, loop=loop)
current.setup(path=chromedriver_path)
current.scroll_covid_table()
current.select_location()
current.start_search()
table_current = current.make_table()

**테이블 확인**

In [8]:
table_current

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,137763,406364,2021-11-18,강동구,-,감염경로조사중,-
1,137761,408143,2021-11-18,강동구,-,기타확진자접촉,-
2,137760,407954,2021-11-18,강동구,-,기타확진자접촉,-
3,137726,406769,2021-11-18,강동구,-,감염경로조사중,-
4,137694,409070,2021-11-18,강동구,-,감염경로조사중,-
...,...,...,...,...,...,...,...
195,133193,398677,2021-11-15,강동구,-,감염경로조사중,-
196,133040,398061,2021-11-15,강동구,-,감염경로조사중,-
197,133039,398329,2021-11-15,강동구,-,기타확진자접촉,-
198,133033,398338,2021-11-15,강동구,-,감염경로조사중,-


**저장**  
**path는 저장하고싶은 경로로 변경**

In [ ]:
path = './covid.xlsx' #(./ 는 소스코드가 있는 현재 위치를 말함)
table_current.to_excel(path) #저장

### 과거 확진자 현황 (1 ~ 80000번)

In [9]:
loop = 2 #반복횟수(얼마만큼의 페이지 데이터를 가져올건지 선택)

old = Old(url = url, loop=loop)
old.setup(path=chromedriver_path)
old.scroll_covid_table()
old.select_location()
old.start_search()
table_old = old.make_table()

**테이블 확인**

In [10]:
table_old

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,80000,251550,2021-08-31,중구,-,감염경로조사중,퇴원
1,79999,253368,2021-08-31,광진구,-,기타확진자접촉,퇴원
2,79998,253369,2021-08-31,광진구,-,기타확진자접촉,퇴원
3,79997,253382,2021-08-31,광진구,-,기타확진자접촉,퇴원
4,79996,253296,2021-08-31,타시도,-,타시도확진자접촉,퇴원
...,...,...,...,...,...,...,...
195,72797,229523,2021-08-18,강동구,-,감염경로조사중,퇴원
196,72784,229520,2021-08-18,강동구,-,감염경로조사중,퇴원
197,72710,229108,2021-08-18,강동구,-,감염경로조사중,퇴원
198,72709,229041,2021-08-18,강동구,-,기타확진자접촉,퇴원


**저장**  
**path는 저장하고싶은 경로로 변경**

In [32]:
path = './covid.xlsx' #(./ 는 소스코드가 있는 현재 위치를 말함)
table_old.to_excel(path) #저장